<a href="https://colab.research.google.com/github/DevClare/Automatic-Scoring-for-Oral-Presentation-using-ML-and-DL/blob/main/CNN%26YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
import pandas as pd
import os

# Mount Google Drive
drive.mount('/content/drive')

# Path to CSV file in the drive
csv_path = "/content/drive/MyDrive/Dataset/AudioScoreLatest.csv"

scores_df = pd.read_csv(csv_path)
print("CSV loaded. Shape:", scores_df.shape)
print(scores_df.head())


Mounted at /content/drive
CSV loaded. Shape: (72, 9)
            Name  Volume  Pace  Tone  Pronounciation  Vocal Variety  \
0  AbdallahTamer       3     4     3               4              3   
1      AlanTiong       5     3     1               4              2   
2         Alsten       2     4     5               3              4   
3    ArielAthomo       5     2     3               2              3   
4      Avvinnesh       4     4     3               5              2   

   Vocal Control  Fluency  Total  
0              4        4    NaN  
1              4        5    NaN  
2              5        5    NaN  
3              3        3    NaN  
4              5        5    NaN  


In [6]:
import numpy as np
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Path to spectrogram images in drive
image_folder = "/content/drive/MyDrive/Dataset/spectrograms" #ADJUST

HUMAN_SCORE_COLUMNS = ["Volume","Pace","Tone","Pronounciation","Vocal Variety","Vocal Control","Fluency"]

#Calculate average of human evaluated scores
scores_df["AvgScore"] = scores_df[HUMAN_SCORE_COLUMNS].mean(axis=1)

# Map average to class, 0 = bad(average score of 1-2), 1 = neutral(average score of 2-3.5), 2 = very good(average score above 3.5)
def map_score_to_class(s):
    if s <= 2: return 0
    elif 2 < s <= 3.5: return 1
    else: return 2

scores_df["Label"] = scores_df["AvgScore"].apply(map_score_to_class)

# Prepare images + labels
IMG_SIZE = 128
image_data, labels = [], []

for idx, row in tqdm(scores_df.iterrows(), total=len(scores_df)):
    filename = row["Name"] + ".jpg"
    filepath = os.path.join(image_folder, filename)
    if os.path.exists(filepath):
        img = cv2.imread(filepath, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        image_data.append(img)
        labels.append(row["Label"])

X = np.array(image_data) / 255.0
y = to_categorical(np.array(labels), num_classes=3)

# Train/test split, 80% for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32)

# Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"CNN Test Accuracy: {test_acc:.2f}")


100%|██████████| 72/72 [00:28<00:00,  2.52it/s]
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 446ms/step - accuracy: 0.4970 - loss: 1.1059 - val_accuracy: 0.5000 - val_loss: 3.6539
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 290ms/step - accuracy: 0.6632 - loss: 2.3710 - val_accuracy: 0.4167 - val_loss: 2.0442
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 272ms/step - accuracy: 0.5370 - loss: 1.3974 - val_accuracy: 0.5000 - val_loss: 1.3906
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 285ms/step - accuracy: 0.7241 - loss: 0.7433 - val_accuracy: 0.4167 - val_loss: 1.2567
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 412ms/step - accuracy: 0.7641 - loss: 0.6171 - val_accuracy: 0.4167 - val_loss: 1.0410
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 409ms/step - accuracy: 0.7729 - loss: 0.6430 - val_accuracy: 0.4167 - val_loss: 1.0334
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 287ms/step - accuracy: 0.8190 - loss: 0.5708 - val_accuracy: 0.3333 - val_loss: 1.3262
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 301ms/step - accuracy: 0.7433 - loss: 0.5595 - val_accuracy: 0.5000 - val_loss:

In [8]:
import shutil
from sklearn.model_selection import train_test_split

# Make YOLO dataset dirs
base_dir = "/content/drive/MyDrive/YourFolder/yolo_dataset"
for split in ["train", "val"]:
    for cls in ["bad", "neutral_good", "very_good"]:
        os.makedirs(os.path.join(base_dir, split, cls), exist_ok=True)

# Split filenames
train_files, val_files = train_test_split(scores_df, test_size=0.2, random_state=42, stratify=scores_df["Label"])

# Helper to map label -> class folder
def label_to_class(lbl):
    return "bad" if lbl == 0 else "neutral_good" if lbl == 1 else "very_good"

# Copy images to YOLO dataset
def copy_files(file_list, split):
    for _, row in tqdm(file_list.iterrows(), total=len(file_list)):
        filename = row["Name"] + ".jpg"
        src = os.path.join(image_folder, filename)
        if os.path.exists(src):
            cls = label_to_class(row["Label"])
            dst = os.path.join(base_dir, split, cls, filename)
            shutil.copy(src, dst)

copy_files(train_files, "train")
copy_files(val_files, "val")


100%|██████████| 15/15 [00:00<00:00, 96.95it/s]


In [9]:
!pip install ultralytics

from ultralytics import YOLO

# Use pretrained YOLOv8 classification model
model = YOLO("yolov8n-cls.pt")

# Train
model.train(
    data=base_dir,
    epochs=20,
    imgsz=128,
    batch=32
)

# Validate
metrics = model.val()
print("YOLOv8 Top-1 Accuracy:", metrics.top1)
print("YOLOv8 Top-5 Accuracy:", metrics.top5)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.6 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.206 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (AMD EPYC 7B12)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/YourFolder/yolo_dataset, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, 